### environment set up 

In [0]:
import pyspark.sql.functions as f 
import pyspark
import os

### utility functions

In [0]:
%run "../../utility_functions/misc"

In [0]:
%run "../../utility_functions/qa_utils"

In [0]:
%run "../../utility_functions/spkdf_utils"

### directories

In [0]:
dir_danny_path = "dbfs:/mnt/feature-store-dev/dev_users/dev_dw/24q4_fs_fundation/"
dir_data_parent_shared = os.path.join(dir_danny_path, "d400_feature/d401_mobile_oa_consumer")
dir_data_parent_mvmt = os.path.join(dir_danny_path, "d500_movement/d501_mobile_oa_consumer")
dir_data_parent_stag = os.path.join(dir_danny_path, "d200_staging/d299_src")

In [0]:
df_fea_coll_action = spark.read.format('delta').load(os.path.join(dir_data_parent_shared ,'fea_coll_action_cycle_12mnth'))
df_fea_product_acq = spark.read.format('delta').load(os.path.join(dir_data_parent_shared, 'fea_product_acquisition_cycle_billing_12'))
df_fea_late_pay = spark.read.format('delta').load(os.path.join(dir_data_parent_shared,"fea_late_pay_cycle_billing_6"))
df_mvmt_aod = spark.read.format('delta').load(os.path.join(dir_data_parent_mvmt, 'mvmt_aod30d'))

# stage table 

df_stg_coll_action = spark.read.format('delta').load(os.path.join(dir_data_parent_stag, "stg_brm_coll_action_hist"))

In [0]:
df_qa_coll_action = spark.read.format('delta').load( '/mnt/ml-lab/dev_users/dev_sc/99_misc/cohort_seg/fea_coll_action')

In [0]:
display(
    df_qa_coll_action
    .groupBy('reporting_date', 'reporting_cycle_type')
    .agg(f.countDistinct('fs_srvc_id')
         , f.countDistinct('poid_id0')
         )
)

### QA

In [0]:
# print('late_pay')
# display(df_fea_late_pay.limit(100))

print('coll_action')
display(df_fea_coll_action.limit(100))

# print('product_acq')
# display(df_fea_product_acq.limit(100))

# print('aod movement')
# display(df_mvmt_aod.limit(100))

#### s01 collection actions 

In [0]:
display(
    df_fea_coll_action
    .groupBy('reporting_date')
    .agg(f.countDistinct('fs_srvc_id')
         , f.count('*')
         , f.count('action_name')
    )
)

In [0]:
ls_test_coll_field_numeric = ['coll_action_category_num', 'total_payment_amt', 'payment_cnt'
                              , 'to_payment_days_avg', 'coll_action_cnt_12mnth', 'pay_after_coll_cnt_12mnth'
                              , 'to_payment_days_avg_12mnth'
                               ]

In [0]:
for i in ls_test_coll_field_numeric:     
    df_result = (
        df_fea_coll_action
        .groupBy('reporting_date')
        .agg(
            f.sum(i).alias('sum'),
            f.mean(i).alias('mean'),
            f.percentile_approx(i, 0.25,100).alias('25pct'),
            f.percentile_approx(i, 0.79,100).alias('75pct'),
            f.percentile_approx(i, 0.95,100).alias('95pct'),
            f.percentile_approx(i, 0.99,100).alias('99pct'),
            f.median(i).alias('median'),
            f.stddev(i).alias('stddev'),
            f.min(i).alias('min'),
            f.max(i).alias('max'), 
            f.countDistinct('fs_srvc_id'), 
            f.count('*') 
            #f.countDistinct('bill_no')
            )
    )
    print(i)
    display(df_result)

In [0]:
display(
    df_fea_coll_action
    .filter(f.col('reporting_date') == '2024-10-31')
    .filter(f.col('action_name').isNotNull())
    .join(
        df_qa_coll_action
        .filter(f.col('reporting_date') == '2024-10-31')
        .filter(f.col('latest_coll_action_name').isNotNull())
        , ['fs_cust_id', 'fs_srvc_id', 'fs_acct_id']
        , 'anti' 
    )
)

In [0]:
display(
    df_qa_coll_action
    .filter(f.col('fs_acct_id') == '416688902')
)  


In [0]:
display(
    df_qa_coll_action
    .groupBy('reporting_date')
    .agg( f.percentile_approx('cnt_in_coll_action_12m', 0.99,100).alias('99pct'))
)


display(
    df_fea_coll_action
    .groupBy('reporting_date')
    .agg(f.percentile_approx('coll_action_cnt_12mnth', 0.99, 100))
)

In [0]:
display(
    df_fea_coll_action
    .filter(f.col('reporting_date') == '2024-10-31')
    .filter(f.col('coll_action_cnt_12mnth') == 92)    
)

In [0]:
display(df_fea_coll_action
        .filter(f.col('fs_acct_id') == '495989274')
        .filter(f.col('reporting_date') == '2024-10-31')
        )

In [0]:
display(
    df_qa_coll_action
    .filter(f.col('reporting_date') == '2024-10-31')
    # .filter(f.col('coll_action_cnt_12mnth') == 777)  
    .filter(f.col('cnt_in_coll_action_12m') ==136  )  
   # .filter(f.col('fs_acct_id') == '490717639')
    # .select('fs_acct_id', 'fs_srvc_id', 'fs_cust_id', 'reporting_date', 'reporting_cycle_type'
    #         , 'coll_action_cnt_12mnth', 'complete_date', 'pay_after_coll_cnt_12mnth', 'coll_action_category'
    #         )
)

In [0]:
display(
    df_stg_coll_action
    .filter(f.col('fs_acct_id') == '490717639')
   # .filter(f.col())
)

In [0]:
display(df_qa_coll_action
        .filter(f.col('reporting_date') == '2024-10-31')
        .filter(f.col('') == '490717639')
        )

In [0]:
display(
    df_fea_coll_action
    .filter(f.col('reporting_date') == '2024-10-31')
    .filter(f.col('coll_action_cnt_12mnth') == '92')
    #.filter(f.col('fs_acct_id') == '490717639')
)

In [0]:
df_result = (
        df_qa_coll_action
        .groupBy('reporting_date')
        .agg(
            f.sum('cnt_pay_after_coll_12m').alias('sum'),
            f.mean('cnt_pay_after_coll_12m').alias('mean'),
            f.percentile_approx('cnt_pay_after_coll_12m', 0.25,100).alias('25pct'),
            f.percentile_approx('cnt_pay_after_coll_12m', 0.79,100).alias('75pct'),
            f.percentile_approx('cnt_pay_after_coll_12m', 0.95,100).alias('95pct'),
            f.percentile_approx('cnt_pay_after_coll_12m', 0.99,100).alias('99pct'),
            f.median('cnt_pay_after_coll_12m').alias('median'),
            f.stddev('cnt_pay_after_coll_12m').alias('stddev'),
            f.min('cnt_pay_after_coll_12m').alias('min'),
            f.max('cnt_pay_after_coll_12m').alias('max'), 
            f.countDistinct('fs_srvc_id'), 
            f.count('*') 
            #f.countDistinct('bill_no')
            )
    )
    
display(df_result)